In [1]:
from deps.dobot_api import DobotApiDashboard, DobotApiMove
from deps import utils
from deps import cv_core
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pynput import keyboard


In [2]:
#Connect to robot sockets. 
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)

In [64]:
dash.DisableRobot()

'0,{},DisableRobot();'

In [61]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [3]:
keys = utils.Keyboard(dash)
cont = cv_core.Contours()

In [4]:
keys.execute()

Position saved!
Position saved!
Position saved!
Position saved!
Special key pressed: Key.esc


In [5]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, (1024, 768))
centers = cont.wait_for_anchors(cap, show = True)
cap.release()

Qt: Session management error: Could not open network socket


In [6]:
centers

[(151, 763), (898, 761), (150, 20), (887, 14)]

In [7]:
xys = [(arr[0], arr[1]) for arr in keys.coords]
pix_coor = utils.assign_corners(centers)
robot_coor = utils.assign_corners(xys, reverse=True)

In [8]:
features_mm_to_pixels_dict = {}
for key, value in robot_coor.items():
    features_mm_to_pixels_dict[value] = pix_coor[key]

In [10]:
tf_mtx = cv_core.compute_tf_mtx(features_mm_to_pixels_dict)

In [11]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, (1024, 768))
#out = cv2.VideoWriter('out.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (640,480))
cont = cv_core.Contours()
while(True):
    ret, frame = cap.read()

    (a,b,r), sorted_contours = cont.find_contours(frame, 5)
    plot_img = frame.copy()
    cv2.circle(plot_img, (a, b), r, (0, 255, 0), 2)
    cv2.circle(plot_img, (a, b), 1, (255, 0, 0), 3)
    #with_contours = cv2.drawContours(plot_img, sorted_contours, -1,(0,255, 0),2)

    for c in sorted_contours:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    #out.write(with_contours)

    cv2.imshow('frame',plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
#out.release()
cv2.destroyAllWindows()

In [12]:
centers_cuboids = cont.contour_centers(sorted_contours)

In [13]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [14]:
utils.default_pos(move)
for center in centers_cuboids:
    X, Y, _ = tf_mtx @ (center[0], center[1], 1)
    move.MovL(X, Y, -70,  0)
    move.MovL(X, Y, -95,  0)
    move.MovL(X, Y, -70,  0)
utils.default_pos(move)

'0,{},JointMovJ(0.000000,0.000000,0.000000,0.000000);'